In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [6]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello,how,are, you?")

['Hello', 'how', 'are', 'you?']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

# prompt = template.format_messages(
#     max_items=10,
#     question="What are the colors?"
# )

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

In [14]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best results.

2. P

AIMessageChunk(content='As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala. Here are some alternative ingredients and preparation methods you can use:\n\n1. **Replace Chicken**: You can substitute the chicken with firm tofu, paneer (Indian cottage cheese), or seitan. Cut them into bite-sized pieces and follow the same marination and baking process as you would with chicken.\n\n2. **Replace Yogurt**: Use plant-based yogurt made from soy, almond, or coconut milk as a substitute for dairy yogurt in the marinade. The tanginess and creaminess will still be present in the dish.\n\n3. **Replace Garam Masala**: To replace the garam masala, you can make your own blend using ground cumin, coriander, cardamom, cinnamon, cloves, and black pepper. Mix these spices in the same proportions as garam masala or adjust to your taste.\n\n4. **Replace Ghee**: Instead of ghee, you can use vegetable oil or vegan butter for cooking the onions and spices. This will